---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
#print(states)

In [3]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g


#'''Returns a DataFrame of towns and the states they are in from the 
#university_towns.txt list. The format of the DataFrame should be:
#DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#columns=["State", "RegionName"]  )
    
#The following cleaning needs to be done:

#1. For "State", removing characters from "[" to the end.
#2. For "RegionName", when applicable, removing every character from " (" to the end.
#3. Depending on how you read the data, you may need to remove newline character '\n'. '''
def get_list_of_university_towns():
    #initiatlize a dataframe struct
    university_towns = pd.DataFrame( [], columns=["State", "RegionName"]  ) 
    #Open the file & prepare to read line by line
    university_towns_filename = 'university_towns.txt'
    university_towns_file = open(university_towns_filename, "r")
    current_state = 'Alabama'
    
    for line in university_towns_file:
         ## If the data line contains "[edit]", it's a state, cache value & continue
        if "edit" in line:
            current_state = line
        ## ELSE add [CACHE, this value] to dataframe
        else:
            #add a row, strip newline
            university_towns.loc[len(university_towns.index)] = [current_state.rstrip(), line.rstrip()] 
    
    #Cleanse data:
    ## Remove "\[.*"
    university_towns['State'] = university_towns['State'].replace(to_replace='\[.*', value='', regex=True)
    ## Remove "\(.*"
    university_towns['RegionName'] = university_towns['RegionName'].replace(to_replace='\(.*', value='', regex=True)
    
    return university_towns


get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [6]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g


'''read in excel, skip the header, skip the first 2 rows, use only 2 cols '''
def get_gdp_dataframe():
    gdp_by_quarter = pd.read_excel(io='gdplev.xlsx', header=5, skiprows=214, usecols=[4,6])
    gdp_by_quarter.columns = [ "quarter" , "gdp" ]
    return gdp_by_quarter
  
    
'''Must have 2 consecutive declining quarters'''
def is_consecutive_quarter_decline(row):
    return ( row['gdp'] < row['prev_gdp'] and row['prev_gdp'] < row['prev_prev_gdp'] )


'''Returns the year and quarter of the recession start time as a 
string value in a format such as 2005q3'''
def get_recession_start():
    #Load-in gdplev.xls
    gdp_by_quarter = get_gdp_dataframe()
    
    #recession == 2 consecutive quarters GDP DECLINE
    #create (2) columns for previous quarters (for comparison)
    gdp_by_quarter['prev_gdp'] = gdp_by_quarter['gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_gdp'] = gdp_by_quarter.loc[0]['gdp']
    gdp_by_quarter['prev_prev_gdp'] = gdp_by_quarter['prev_gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_prev_gdp'] = gdp_by_quarter.loc[0, 'prev_gdp']

    #Now simply create mask where gdp < prev & prev < prev-prev 
    gdp_recession_mask = gdp_by_quarter.apply(is_consecutive_quarter_decline, axis=1)
    gdp_recession_start_quarters = gdp_by_quarter[gdp_recession_mask]
    
    #The recession start will be the first occurence of this phenomenom
    recession_start_idx = gdp_recession_start_quarters.index[0]
    recession_start = gdp_recession_start_quarters.loc[recession_start_idx, 'quarter']

    return (recession_start)


get_recession_start()

'2008q4'

In [8]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g


'''Must have 2 consecutive increasing quarters'''
def is_consecutive_quarter_increase(row):
    return ( row['gdp'] > row['prev_gdp'] and row['prev_gdp'] > row['prev_prev_gdp'] )


'''Returns the year and quarter of the recession end time as a 
string value in a format such as 2005q3'''
def get_recession_end():
    #Load-in gdplev.xls
    gdp_by_quarter = get_gdp_dataframe()
    
    #Search for recession end (recession_end == 2 consecutive quarters GDP INCREASE)
    #create (2) columns for previous quarters (for comparison)
    gdp_by_quarter['prev_gdp'] = gdp_by_quarter['gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_gdp'] = gdp_by_quarter.loc[0]['gdp']
    gdp_by_quarter['prev_prev_gdp'] = gdp_by_quarter['prev_gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_prev_gdp'] = gdp_by_quarter.loc[0, 'prev_gdp']

    #Get recession start & slice data to ONLY have this data
    recession_start = get_recession_start()
    recession_start_row = gdp_by_quarter[gdp_by_quarter['quarter'] == recession_start]
    recession_start_index = recession_start_row.index[0]
    gdp_by_quarter_after_recession_start = \
        gdp_by_quarter.iloc[recession_start_index:(gdp_by_quarter.index[-1] + 1)]

    #Now simply create mask where gdp < prev & prev < prev-prev 
    gdp_recession_mask = gdp_by_quarter_after_recession_start.apply(is_consecutive_quarter_increase, axis=1)
    gd_recession_end_quarters = gdp_by_quarter_after_recession_start[gdp_recession_mask]
    
    #The recession start will be the first occurence of this phenomenom
    recession_end_idx = gd_recession_end_quarters.index[0]
    recession_end = gd_recession_end_quarters.loc[recession_end_idx, 'quarter']

    return (recession_end)

get_recession_end()



'2009q4'

In [32]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g

'''Returns the year and quarter of the recession bottom time as a 
string value in a format such as 2005q3'''
def get_recession_bottom():
    #Get GDP
    gdp_by_quarter = get_gdp_dataframe()
    #Get the start
    start_quarter = get_recession_start()
    #Get the end
    end_quarter = get_recession_end()
    #Find the slice of rows in between
    recession_start_row = gdp_by_quarter[gdp_by_quarter['quarter'] == start_quarter]
    recession_start_index = recession_start_row.index[0]
    recession_end_row = gdp_by_quarter[gdp_by_quarter['quarter'] == end_quarter]
    recession_end_index = recession_end_row.index[0]
    gdp_by_quarter_recession = gdp_by_quarter.iloc[recession_start_index:\
                                                  recession_end_index]
    gdp_by_quarter_recession = \
        gdp_by_quarter.iloc[recession_start_index:(recession_end_index+1)]
    
    #min_quarter_gdp = gdp_by_quarter_recession[ \
            #gdp_by_quarter_recession.where(\
            #gdp_by_quarter_recession['gdp'] == \
            #(gdp_by_quarter_recession['gdp'].min()))]
    
    min_quarter_gdp = gdp_by_quarter_recession.where(\
            gdp_by_quarter_recession['gdp'] == \
            (gdp_by_quarter_recession['gdp'].min()))
    min_quarter_gdp = min_quarter_gdp.dropna()
    
    return (min_quarter_gdp['quarter'].iloc[0])

get_recession_bottom()

'2009q2'

In [71]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g

'''Use the state abbreviation map to transfer from abbreviation to full name'''
def replace_state_abbreviation_with_full_name(row):
    row['State'] = states[row['State']]
    return (row)


'''Combine 3 month segments {1:{01-03}, 2:{04-06}, 3:{07-09}, 4:{10-12}}'''
def combine_months_into_quarters(row):
    return (row)

'''Converts the housing data to quarters and returns it as mean 
values in a dataframe. This dataframe should be a dataframe with
columns for 2000q1 through 2016q3, and should have a multi-index
in the shape of ["State","RegionName"].

Q1 is {01 - 03}, January through March, 
Q2 is {04 - 06}, April through June, 
Q3 is {07 - 09}, July through September, 
Q4 is {10 - 12}, October through December.
    
The resulting dataframe should have 67 columns, and 10,730 rows.
'''
def convert_housing_data_to_quarters():
    #Step 1: read the excel file as a dataframe
    base_city_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    #Map all state acronyms to actual state names
    city_data = base_city_data.apply(replace_state_abbreviation_with_full_name, axis=1)
    
    #Setup multi-index as [State (col 2), RegionName (col 1)]
    city_data = city_data.set_index(['State', 'RegionName']).sort_index()
    
    # Strip useless columns
    city_data = city_data.iloc[:, 49:]
        
    #Strip all excess columns: keep 2000q1 - 2016q3
    ## Get slice of dataframe using df['2000q1':'2016q3']
    
    
    

    
    #Combine months into quarters
    #Effectively apply a function to each row?
    
    return (city_data)

convert_housing_data_to_quarters()

2000-01   2000-02   2000-03   2000-04   2000-05  \
State   RegionName                                                      
Alabama Adamsville    68700.0   69100.0   69300.0   69200.0   69100.0   
        Alabaster    121600.0  122300.0  122500.0  122800.0  123200.0   
        Albertville   74300.0   74100.0   73500.0   72700.0   72500.0   
        Arab          84500.0   83900.0   82900.0   81900.0   81400.0   
        Ardmore           NaN       NaN       NaN       NaN       NaN   
...                       ...       ...       ...       ...       ...   
Wyoming Burns        100900.0  101300.0  102400.0  103600.0  104400.0   
        Casper        88400.0   89300.0   90000.0   90100.0   89600.0   
        Cheyenne     116000.0  116700.0  117900.0  119200.0  120100.0   
        Evansville   128500.0  128100.0  127500.0  127700.0  128600.0   
        Pine Bluffs   93900.0   93400.0   93900.0   94900.0   95300.0   

                      2000-06   2000-07   2000-08   2000-09   2000-10  ...  \
State   RegionName                                                     ...   
Alabama Adamsville    69200.0   69500.0   69700.0   70200.0   71100.0  ...   
        Alabaster    123200.0  123100.0  123100.0  123300.0  123500.0  ...   
        Albertville   72600.0   72500.0   72800.0   73200.0   73500.0  ...   
        Arab          81400.0   81200.0   81200.0   81600.0   82400.0  ...   
        Ardmore           NaN       NaN       NaN       NaN       NaN  ...   
...                       ...       ...       ...       ...       ...  ...   
Wyoming Burns        105700.0  107000.0  108200.0  109900.0  111700.0  ...   
        Casper        89100.0   89000.0   89500.0   90700.0   92100.0  ...   
        Cheyenne     120800.0  121000.0  121400.0  122200.0  123000.0  ...   
        Evansville   130000.0  130900.0  130600.0  131000.0  132300.0  ...   
        Pine Bluffs   95000.0   94100.0   94200.0   95600.0   97200.0  ...   

                     2015-11  2015-12  2016-01  2016-02  2016-03  2016-04  \
State   RegionName                                                          
Alabama Adamsville     73900    73500    73600    74100    74800    75200   
        Alabaster     151800   152400   152800   153400   154200   154900   
        Albertville    91600    91500    91400    92000    92600    92600   
        Arab          112500   112100   111000   109800   109300   109500   
        Ardmore       144200   143100   142200   142500   143000   143200   
...                      ...      ...      ...      ...      ...      ...   
Wyoming Burns         164100   164500   166800   168500   169500   171100   
        Casper        182300   182600   184000   184500   184500   184900   
        Cheyenne      190500   191900   193800   194700   194800   195500   
        Evansville    303900   306600   308200   309700   310400   310200   
        Pine Bluffs   169600   169300   168400   166300   165800   166600   

                     2016-05  2016-06  2016-07  2016-08  
State   RegionName                                       
Alabama Adamsville     74900    74700    74700    74700  
        Alabaster     155200   155200   155600   156100  
        Albertville    92400    92400    92300    92100  
        Arab          110100   110700   111600   112400  
        Ardmore       143400   143100   142300   141600  
...                      ...      ...      ...      ...  
Wyoming Burns         172100   171600   170700   170300  
        Casper        185400   185200   184600   184100  
        Cheyenne      196400   197600   198800   199400  
        Evansville    308800   308600   308800   309300  
        Pine Bluffs   164500   160300   158000   157300  

[10730 rows x 200 columns]

In [ ]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g



def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

run_ttest()